In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import shap 

In [4]:
train_path = "../data/local_train.csv"
test_path = "../data/local_test.csv"

In [5]:
from lightgbm import LGBMClassifier

In [6]:
import hashlib

In [7]:
s="A"
int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**2

45

In [8]:
def hash_cat(s):
    return int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**3

In [9]:
# x = ['A', "b"]


In [10]:
# x.index('c')

In [11]:
cat_dict = []
def hash_cat_two(s):
    if s not in cat_dict:
        cat_dict.append(s)
    return cat_dict.index(s)

In [12]:
import pandas as pd

In [13]:
!ls ../data/

local_test.csv	local_train.csv  sample_submission.csv	test.csv  train.csv


In [14]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)


In [15]:
len(df_train)

180000

In [16]:
df_train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,4700,B,F,A,B,E,BQ,A,N,BL,...,0.236103,0.316480,0.600188,0.192628,0.423268,0.176169,0.288662,0.793823,0.315937,0
1,453468,A,F,G,C,G,BI,A,U,AE,...,0.627595,0.657871,0.285636,0.735430,0.624774,0.624786,0.702093,0.305428,0.509672,0
2,369751,A,K,G,A,D,BU,A,E,Y,...,0.568753,0.790244,0.829496,0.015555,0.201568,0.786024,0.385816,0.431418,0.953504,0
3,69873,B,I,A,A,E,BI,A,Y,AU,...,0.775825,0.822142,0.796378,0.680048,0.246919,0.877006,0.611704,0.839066,0.934136,0
4,170717,A,M,A,B,F,BI,I,AH,BM,...,0.256564,0.579400,0.291327,0.737406,0.579729,0.290138,0.351962,0.269911,0.308573,1


In [17]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [18]:
df_train.columns

Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'target'],
      dtype='object')

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
# ?train_test_split

In [21]:
cat_col = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18']
cont_col = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4',
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10']

In [22]:
for col in cat_col:
    encoder = LabelEncoder()
    # encoder.fit(list(df_train[col]) + list(df_test[col]))
    encoder.fit(list(df_train[col].values) + list(df_test[col].values))
    df_train[col] = encoder.transform(df_train[col].values)
    df_test[col] = encoder.transform(df_test[col].values)
    print("done ", col)

done  cat0
done  cat1
done  cat2
done  cat3
done  cat4
done  cat5
done  cat6
done  cat7
done  cat8
done  cat9
done  cat10
done  cat11
done  cat12
done  cat13
done  cat14
done  cat15
done  cat16
done  cat17
done  cat18


In [23]:
X = df_train.drop(["id", "target"], axis=1)
y = df_train["target"]


In [24]:
X.shape

(180000, 30)

In [25]:
y.shape

(180000,)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=111)

In [27]:
import lightgbm as lgb

In [28]:
df_train["cat0"].unique()

array([1, 0])

In [29]:
# train_data = gbm.Dataset(X_train, label=y_train, categorical_feature=cat_col)

In [30]:
# validation_data = gbm.Dataset(X_test, label=y_test, reference=train_data)

In [31]:
# lgbm_params = {
#     'boosting': 'gbdt',          # dart (drop out trees) often performs better
#     'application': 'binary',     # Binary classification
#     'learning_rate': 0.02,       # Learning rate, controls size of a gradient descent step
#     'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
#     'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
#     'num_leaves': 100,            # Controls size of tree since LGBM uses leaf wise splits
#     'metric': 'auc',  # Area under ROC curve as the evaulation metric
# }

https://www.kaggle.com/sayantansadhu/lgbm-with-hyper-parameter-tuning-using-optuna

In [57]:
lgbm_params = {
    'boosting': 'gbdt',          # dart (drop out trees) often performs better
    'objective': 'binary',     # Binary classification
    'learning_rate': 0.02,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 100,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'metric': 'binary_logloss',  # Area under ROC curve as the evaulation metric,
    'lambda_l1': 5e-05, 
     'lambda_l2': 1.35e-08, 
     'num_leaves': 216, 
     'feature_fraction': 0.7458519562366559, 
     'bagging_fraction': 0.7835116194444349, 
     'bagging_freq': 2, 
     'min_child_samples': 95,
    'n_estimators':300
}

# {'lambda_l1': 5.7529066102502985e-05, 
#  'lambda_l2': 1.3522520634637266e-08, 
#  'num_leaves': 216, 
#  'feature_fraction': 0.7458519562366559, 
#  'bagging_fraction': 0.7835116194444349, 
#  'bagging_freq': 2, 
#  'min_child_samples': 95}

In [58]:
train_data = lgb.Dataset(X_train, label=y_train)

In [59]:
validation_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [60]:
num_round = 1000
model = lgb.train(lgbm_params, train_data, num_round, valid_sets=[validation_data], early_stopping_rounds=10, verbose_eval=100)

Found `n_estimators` in params. Will use it instead of argument


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 43102, number of negative: 118898
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058491 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3294
[LightGBM] [Info] Number of data points in the train set: 162000, number of used features: 30
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=95 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.266062 -> initscore=-1.014697
[LightGBM] [Info] Start training from score -1.014697
Training until validation scores don't improve for 10 rounds
[100

In [61]:
df_test2 = df_test.drop(["id", "target"], axis=1)

In [62]:
df_test2.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,0,1,0,1,7,33,6,31,0,0,...,0.832036,0.936027,0.277879,0.677056,0.556303,0.861982,0.880029,0.700449,0.408714,0.457853
1,0,8,0,0,3,2,12,8,12,0,...,0.461403,0.465581,0.773081,0.257367,0.118800,0.076394,0.617704,0.786615,0.463401,0.534980
2,0,5,0,0,4,33,0,32,60,0,...,0.727531,0.799496,0.792766,0.574415,0.240725,0.263741,1.005591,0.710779,0.878274,0.967304
3,1,8,5,1,5,33,0,14,45,0,...,0.294513,0.301259,0.340367,0.385370,0.555850,0.486549,0.338415,0.392193,0.255491,0.398400
4,0,10,0,0,4,33,2,26,38,5,...,0.354699,0.317978,0.590793,0.207941,0.734811,0.569722,0.569788,0.477121,0.530924,0.349163


In [63]:
pred = model.predict(df_test2)

In [64]:
df_test2

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,0,1,0,1,7,33,6,31,0,0,...,0.832036,0.936027,0.277879,0.677056,0.556303,0.861982,0.880029,0.700449,0.408714,0.457853
1,0,8,0,0,3,2,12,8,12,0,...,0.461403,0.465581,0.773081,0.257367,0.118800,0.076394,0.617704,0.786615,0.463401,0.534980
2,0,5,0,0,4,33,0,32,60,0,...,0.727531,0.799496,0.792766,0.574415,0.240725,0.263741,1.005591,0.710779,0.878274,0.967304
3,1,8,5,1,5,33,0,14,45,0,...,0.294513,0.301259,0.340367,0.385370,0.555850,0.486549,0.338415,0.392193,0.255491,0.398400
4,0,10,0,0,4,33,2,26,38,5,...,0.354699,0.317978,0.590793,0.207941,0.734811,0.569722,0.569788,0.477121,0.530924,0.349163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,1,8,0,0,5,33,4,40,35,0,...,0.460675,0.690114,0.556406,0.778571,0.280025,0.544267,0.524250,0.502416,0.782653,0.683392
119996,1,10,0,4,7,33,0,30,38,0,...,0.702119,0.698665,0.771236,0.806725,0.556306,0.339435,0.740020,0.420333,0.957234,0.777523
119997,0,10,0,0,4,33,2,39,38,7,...,0.325691,0.245823,0.243893,0.808406,0.273086,0.577464,0.283429,0.347557,0.254384,0.269322
119998,0,7,0,0,5,45,7,28,3,5,...,0.621029,0.940450,0.082610,0.238615,0.274609,0.425721,0.321446,0.323698,0.525235,0.796209


In [65]:
df_pred_final = df_test[["id"]]

In [66]:
df_pred_final["target"] = pred


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [68]:
# pred_proba[:,0]

In [69]:
df_pred_final.head()

,id,target
0,99402,0.968620
1,92026,0.054544
2,102280,0.055327
3,293468,0.023322
4,56800,0.226151


# df_pred_final.to_csv('submission.csv', index=False)

In [70]:
pred

array([0.96862013, 0.05454422, 0.05532735, ..., 0.10843397, 0.60890904,
       0.70280209])

# evaluation local

In [71]:
from sklearn.metrics import roc_auc_score

In [72]:
y_test_label = df_test["target"]

In [73]:
roc_auc_score(y_test_label, pred)

0.8915191600858575

In [74]:
# local test result 
auc = 0.8969435887822527

In [75]:
# roc_auc_score(y_test_label, pred_proba[:,1])

# run shap

In [76]:
import shap

In [77]:
# explainer = shap.Explainer(model)
# shap_values = explainer(X)

In [78]:
%%time
tree_explainer = shap.TreeExplainer(model)


CPU times: user 2.31 s, sys: 32.1 ms, total: 2.34 s
Wall time: 2.34 s


In [79]:
# %%time
# explainer = shap.TreeExplainer(lgbm_model)
# shap_values = explainer.shap_values(train_features[feature_names])
# shap_values_1 = shap_values[1]
# shap.initjs()

In [80]:
X_sample = X.sample(1000)

In [92]:
%%time
shape_value = tree_explainer.shap_values(X_sample)

CPU times: user 57.1 s, sys: 124 ms, total: 57.3 s
Wall time: 17.4 s


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [93]:
type(shape_value)

list

In [95]:
type(tree_explainer)

shap.explainers._tree.Tree

In [98]:
shap.force_plot(tree_explainer.expected_value[1], shape_value[1][0,:], X.iloc[0,:])

In [99]:
shap.force_plot(tree_explainer.expected_value[1], shape_value[1][:,:], X.iloc[:,:])

In [97]:
type(tree_explainer.expected_value[1])

numpy.float64

In [90]:
type(tree_explainer.expected_value[1])

numpy.float64